<a href="https://colab.research.google.com/github/gos-apoorv/apache-airflow/blob/main/Stored_Model_Logistic_Regression_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount drive
from google.colab import drive
content_path = '/content/drive'
drive.mount(content_path)
drive_path = content_path + '/MyDrive/geocoding'


Mounted at /content/drive


In [ ]:
%pip -qq install diskcache 
import diskcache as dc

import os
from tqdm.autonotebook import tqdm

import json
from google.colab import output
output.enable_custom_widget_manager()


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd



!du -h --max-depth=1 {drive_path}/cache
cache = dc.Cache(drive_path+'/cache')
cache.count

     |████████████████████████████████| 44 kB 2.9 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


4.3M	/content/drive/MyDrive/geocoding/cache


1102

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# BQ client for BQ data queries
from google.cloud import bigquery
bq_client = bigquery.Client(project="gelato-data-warehouse")

Authenticated


In [ ]:
from hashlib import md5
import pandas as pd


def get_cached_df_path(cid):
  return f"{content_path}/MyDrive/pandas_cache/{cid}.pk"

def get_cached_df(cid):
  path = get_cached_df_path(cid)
  if os.path.exists(path):
    return pd.read_pickle(path)
  else:
    return None

def set_cached_df(cid, df):
  path = get_cached_df_path(cid)
  os.makedirs(os.path.dirname(path), exist_ok=True)
  df.to_pickle(path)

def cached_query(query:str, client: bigquery.client, cache: bool = True, cid: str = None) -> pd.DataFrame:
  if cid is None:
    cid = md5(query.encode()).hexdigest()

  if cache:
    cached_df = get_cached_df(cid)
    if cached_df is not None:
      return cached_df

  query_job = client.query(query)
  df = query_job.result().to_dataframe()

  if cache:
    set_cached_df(cid, df)

  return df

# Get Data For Prediction

In [ ]:
df_prediction = cached_query("""
WITH network_print_job AS (select job.*, package.client_id
      from pgsql_printcloud_view.public_print_job as job
      join pgsql_printcloud_view.public_package as package
        on job.package_id = package.package_id
      where
        date(job.created_at) >=
        date_trunc(date_add(current_date(), interval -2 year), year)

    )
  ,  platform_product AS (select
        product.*,
        replace(model.description, ' product', '') as commercial_product

      from pgsql_product_platform_view.list_product as product
      left join pgsql_product_platform_view.model_model as model
        on product.product_model_uid = model.id

    )
  ,  network_product_mapping_groups AS (select distinct
        product.product_uid,
        pgroups.product_group

      from platform_product as product
      join google_sheets_us.network_product_mapping_groups as pgroups
        on product.product_uid like concat(pgroups.product_uid_starts, '%')

    )
  ,  network_print_job_event AS (select
        hist.*,
        curr.last_event_at,
        package.order_date as ordered_at,
        package_dates.promised_min_dispatch_date as min_dispatch_promised_at,
        package_dates.promised_delivery_max_date as delivery_max_promised_at,
        package_dates.promised_delivery_min_date as delivery_min_promised_at

      from (
        select
          print_job_id,
          max(case when status_id = 16 then created_at else null end) as uploading_to_storage_at,
          max(case when status_id = 3 then created_at else null end) as failed_at,
          max(case when status_id = 1 then created_at else null end) as new_at,
          max(case when status_id = 4 then created_at else null end) as prepared_at,
          max(case when status_id = 5 then created_at else null end) as printed_at,
          max(case when status_id = 6 then created_at else null end) as shipped_at,
          min(case when status_id = 6 then created_at else null end) as scanned_at,
          max(case when status_id = 17 then created_at else null end) as cancelled_at,
          min(case when status_id = 19 then created_at else null end) as in_transit_at,
          max(case when status_id = 21 then created_at else null end) as delivered_at

          from pgsql_printcloud_view.public_print_job_status_log
          group by 1
      ) as hist

      left join (
        select
          print_job_id,
          created_at as last_event_at
            from (
              select
                print_job_id,
                created_at,
                row_number() over (partition by print_job_id order by created_at desc) as event_order

              from pgsql_printcloud_view.public_print_job_status_log
            )
        where event_order = 1
      ) as curr

      on hist.print_job_id = curr.print_job_id

      left join pgsql_printcloud_view.public_print_job as job
        on hist.print_job_id = job.print_job_id

      left join pgsql_printcloud_view.public_package as package
        on job.package_id = package.package_id

      left join pgsql_printcloud_view.public_package_dispatch_date as package_dates
        on package.package_id = package_dates.package_id

    )
  ,  network_package AS (select
        package.*,
        security_client.client_type

      from pgsql_printcloud_view.public_package as package
      left join pgsql_printcloud_view.security_client as security_client
        on package.client_id = security_client.client_id

    )
  ,  tracking_package AS (select * from tracking_service.tracking_package_tracking
      where DeletedAt is null

    )
  ,  tracking_trail AS (select * from tracking_service.tracking_audit_trail
      where DeletedAt is null

    )
  ,  tracking_status_log AS (select
        log.*,
        packages.PackageID,
        case log.StatusUID
          when 0 then 'Registered'
          when 1 then 'In Transit'
          when 2 then 'Out for Delivery'
          when 3 then 'Failed Attempt'
          when 4 then 'Shipping Exception'
          when 5 then 'Available for Pickup'
          when 6 then 'Delivered'
          when 7 then 'Dispatched'
          when 8 then 'Returned'
          else 'Other'
        end as Status,
        ROW_NUMBER() over (PARTITION BY packages.PackageID ORDER BY log.ThirdPartyEventAt) as log_rank,
        case
        when DATE_DIFF(lead(log.ThirdPartyEventAt,1) OVER (PARTITION BY packages.PackageID ORDER BY log.ThirdPartyEventAt),log.ThirdPartyEventAt,second)/3600 is null and log.StatusUID !=6
          then timestamp_diff(current_timestamp(),log.ThirdPartyEventAt,hour)
        else DATE_DIFF(lead(log.ThirdPartyEventAt,1) OVER (PARTITION BY packages.PackageID ORDER BY log.ThirdPartyEventAt),log.ThirdPartyEventAt,second)/3600
        end as time_spent
          from tracking_service.tracking_package_tracking as packages
        join tracking_service.tracking_audit_trail as trails
          on packages.ID = trails.PackageTrackingID and
             trails.DeletedAt is null
        join tracking_service.tracking_audit_trail_status_log as log
          on trails.ID = log.AuditTrailID and
             log.DeletedAt is null

    )
  ,  tracking_status_log_min_delivered_date AS (select
            a.PackageID,
            a.min_event_date_delivered
          from
          (select
            PackageID,
            min(ThirdPartyEventAt) as min_event_date_delivered
           from tracking_status_log
            where
              Status = 'Available for Pickup' OR Status = 'Delivered'
             group by 1) a )
  ,  tracking_status_log_min_date_in_transit AS (select
            a.PackageID,
            a.min_event_date_in_transit
          from
          (select
            PackageID,
            min(ThirdPartyEventAt) as min_event_date_in_transit
           from tracking_status_log
            where
              Status = 'In Transit'
             group by 1) a )
  ,  tracking_status_log_min_date_out_for_delivery AS (select
            a.PackageID,
            a.min_event_date_out_for_delivery
          from
          (select
            PackageID,
            min(ThirdPartyEventAt) as min_event_date_out_for_delivery
           from tracking_status_log
            where
              Status = 'Out for Delivery'
             group by 1) a )
  ,  shipment_method AS (select *
      from pgsql_shipment_service_view.public_methods
      --where is_enabled
      --DBI 1236: blanks were coming up for previous orders
    )
  ,  shipment_zone_postcode_range AS (select
        p.*,
        z.country_id,
        z.has_postcode_range,

        --06-Apr-2022 Mahesh: Added for transit time analysis
        c.title         as print_house,
        d.title         as shipment_method_name,
        e.title         as shipment_provider_name

      from pgsql_shipment_service_view.prices_shipment_zone_postcode_range as p
      left join pgsql_shipment_service_view.prices_shipment_country_zone as z
      on p.shipment_country_zone_id = z.shipment_country_zone_id

      left join pgsql_shipment_service_view.prices_shipment_delivery_information as b
      on z.shipment_country_zone_id = b.shipment_country_zone_id

      left join pgsql_shipment_service_view.public_locations as c on
      b.location_id = c.location_id

      left join pgsql_shipment_service_view.public_methods as d on
      b.method_id = d.method_id

      left join pgsql_shipment_service_view.public_providers as e on
      d.provider_id = e.provider_id
    )
  ,  shipment_country_zone AS (select
        a.*,
        c.title         as print_house,
        d.title         as shipment_method_name,
        e.title         as shipment_provider_name,
        f.iso_code      as Zone_Country_Id
      from
          pgsql_shipment_service_view.prices_shipment_country_zone as a
          left join pgsql_shipment_service_view.prices_shipment_delivery_information as b
          on a.shipment_country_zone_id = b.shipment_country_zone_id

          left join pgsql_shipment_service_view.public_locations as c on
          b.location_id = c.location_id

          left join pgsql_shipment_service_view.public_methods as d on
          b.method_id = d.method_id

          left join pgsql_shipment_service_view.public_providers as e on
          d.provider_id = e.provider_id

          left join pgsql_shipment_service_view.public_countries as f on
          a.country_id = f.country_id
    )
SELECT
    (MOD((EXTRACT(DAYOFWEEK FROM TIMESTAMP(FORMAT_TIMESTAMP('%F %X', timestamp(datetime(network_package.order_date),'UTC'), network_print_house.time_zone)) ) - 1) - 1 + 7, 7)) AS network_package_order_local_day_of_week_index,
        (FORMAT_TIMESTAMP('%A', TIMESTAMP(FORMAT_TIMESTAMP('%F %X', timestamp(datetime(network_package.order_date),'UTC'), network_print_house.time_zone)) )) AS network_package_order_local_day_of_week,
    network_print_house.name  AS network_print_house_name,
    shipment_country_zone.shipment_method_name  AS shipment_country_zone_zone_shipment_method_name,
    (case

        when network_product_mapping_groups.product_uid like 'flat_product%' then
          if(
             network_product_mapping_groups.product_uid like '%_a5_%' or
             network_product_mapping_groups.product_uid like '%_a6_%' or
             network_product_mapping_groups.product_uid like '%_sx_%' or
             network_product_mapping_groups.product_uid like '%_lt_%' or
             network_product_mapping_groups.product_uid like '%_dl_%' or
             network_product_mapping_groups.product_uid like '%_a4_pt_90-gsm%' or
             network_product_mapping_groups.product_uid like '%_a4_pt_150-gsm%' or
             network_product_mapping_groups.product_uid like '%_a4_pt_300-gsm%' or
             network_product_mapping_groups.product_uid like '%sm_pt_80-lb%' or
             network_product_mapping_groups.product_uid like '%sm_pt_100-lb%' or
             network_product_mapping_groups.product_uid like '%sm_pt_130-lb%' or
             network_product_mapping_groups.product_uid like '%5r_pt_100-lb%' or
             network_product_mapping_groups.product_uid like '%5r_pt_110-lb%' or
             network_product_mapping_groups.product_uid like '%5r_pt_130-lb%' or
             network_product_mapping_groups.product_uid like '%5r_pt_90-gsm%' or
             network_product_mapping_groups.product_uid like '%5r_pt_150-gsm%' or
             network_product_mapping_groups.product_uid like '%5r_pt_300-gsm%' or
             network_product_mapping_groups.product_uid like '%_4x9-inch_%' or
             network_product_mapping_groups.product_uid like '%_140x180-mm_%' or
             network_product_mapping_groups.product_uid like '%_sq_%' or
             network_product_mapping_groups.product_uid like '%_bx_%',
             'Cards',
            'Posters'
          )

        when network_product_mapping_groups.product_uid like 'flat_product%' then
          if(
            network_product_mapping_groups.product_uid like '%170-gsm%' or
            network_product_mapping_groups.product_uid like '%200-gsm%' or
            network_product_mapping_groups.product_uid like '%250-gsm-uncoated-offwhite-archival%' or
            network_product_mapping_groups.product_uid like '%65-lb%' or
            network_product_mapping_groups.product_uid like '%80-lb%' or
            network_product_mapping_groups.product_uid like '%100-lb-cover-uncoated-offwhite-archival%',
            'Posters',
            'Cards'
          )

        when network_product_mapping_groups.product_uid like 'flyers%' then
          if(
            network_product_mapping_groups.product_uid like '%_a5_%' or
            network_product_mapping_groups.product_uid like '%_dl_%' or
            network_product_mapping_groups.product_uid like '%_a6_%' or
            network_product_mapping_groups.product_uid like '%_a4_pt_90-gsm%' or
            network_product_mapping_groups.product_uid like '%_a4_pt_150-gsm%' or
            network_product_mapping_groups.product_uid like '%_a4_pt_300-gsm%',
             'Cards',
            'Posters'
          )

        when network_product_mapping_groups.product_uid like 'flyers%' then
          if(
            network_product_mapping_groups.product_uid like '%170-gsm%' or
            network_product_mapping_groups.product_uid like '%200-gsm%' or
            network_product_mapping_groups.product_uid like '%250-gsm-uncoated-offwhite-archival%' or
            network_product_mapping_groups.product_uid like '%65-lb%' or
            network_product_mapping_groups.product_uid like '%80-lb%' or
             network_product_mapping_groups.product_uid like '%100-lb-cover-uncoated-offwhite-archival%',
            'Posters',
            'Cards'
          )

        when network_product_mapping_groups.product_uid like 'cards%' then
          if(
            network_product_mapping_groups.product_uid like '%170-gsm%' or
            network_product_mapping_groups.product_uid like '%200-gsm%' or
            network_product_mapping_groups.product_uid like '%250-gsm-uncoated-offwhite-archival%' or
            network_product_mapping_groups.product_uid like '%65-lb%' or
            network_product_mapping_groups.product_uid like '%80-lb%' or
             network_product_mapping_groups.product_uid like '%100-lb-cover-uncoated-offwhite-archival%',
            'Posters',
            'Cards'
          )

        else ifnull(network_product_mapping_groups.product_group, 'Other')

      end) AS network_product_mapping_groups_product_group,
        (DATE(case

        when

          (network_print_job_event.shipped_at = network_print_job_event.last_event_at)

          or

          (network_print_job_event.shipped_at >= network_print_job_event.printed_at)

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.shipped_at >= network_print_job_event.prepared_at))

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.prepared_at is null))

          or

          (network_print_job_event.shipped_at <= network_print_job_event.delivered_at)

          or

          (network_print_job_event.shipped_at <= network_print_job_event.in_transit_at)

        then network_print_job_event.shipped_at

        else null

      end

    )) AS network_print_job_event_shipped_date,
    shipment_method.shipping_type  AS shipment_method_shipping_type,
    shipment_country_zone.name  AS shipment_country_zone_zone_name,
    network_shipment_data_country.emoji  AS network_shipment_data_country_emoji,
    network_shipment_data_country.country_name  AS network_shipment_data_country_country_name,
        (MOD((EXTRACT(DAYOFWEEK FROM case

        when

          (network_print_job_event.shipped_at = network_print_job_event.last_event_at)

          or

          (network_print_job_event.shipped_at >= network_print_job_event.printed_at)

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.shipped_at >= network_print_job_event.prepared_at))

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.prepared_at is null))

          or

          (network_print_job_event.shipped_at <= network_print_job_event.delivered_at)

          or

          (network_print_job_event.shipped_at <= network_print_job_event.in_transit_at)

        then network_print_job_event.shipped_at

        else null

      end

    ) - 1) - 1 + 7, 7)) AS network_print_job_event_shipped_day_of_week_index,
        (FORMAT_TIMESTAMP('%A', case

        when

          (network_print_job_event.shipped_at = network_print_job_event.last_event_at)

          or

          (network_print_job_event.shipped_at >= network_print_job_event.printed_at)

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.shipped_at >= network_print_job_event.prepared_at))

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.prepared_at is null))

          or

          (network_print_job_event.shipped_at <= network_print_job_event.delivered_at)

          or

          (network_print_job_event.shipped_at <= network_print_job_event.in_transit_at)

        then network_print_job_event.shipped_at

        else null

      end

    )) AS network_print_job_event_shipped_day_of_week,
        (EXTRACT(HOUR FROM case

        when

          (network_print_job_event.shipped_at = network_print_job_event.last_event_at)

          or

          (network_print_job_event.shipped_at >= network_print_job_event.printed_at)

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.shipped_at >= network_print_job_event.prepared_at))

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.prepared_at is null))

          or

          (network_print_job_event.shipped_at <= network_print_job_event.delivered_at)

          or

          (network_print_job_event.shipped_at <= network_print_job_event.in_transit_at)

        then network_print_job_event.shipped_at

        else null

      end

    )) AS network_print_job_event_shipped_hour_of_day,
    case
        when ('free-customs' in UNNEST(shipment_delivery_terms.tag_list)) = true then 'Free'
        when ('customs-risk' in UNNEST(shipment_delivery_terms.tag_list)) = true then 'DDP with risk'
        when (shipment_delivery_terms.payment_type is not null and shipment_delivery_terms.payment_type = 'DDP') = true then 'DDP'
        when (shipment_delivery_terms.payment_type is not null and shipment_delivery_terms.payment_type = 'DDU' and not ('free-customs' in UNNEST(shipment_delivery_terms.tag_list))) = true then 'DDU'
        else null
      end
     AS shipment_delivery_terms_delivery_terms,
        (DATE(tracking_status_log_min_date_out_for_delivery.min_event_date_out_for_delivery )) AS tracking_status_log_min_date_out_for_delivery_out_for_delivery_min_date,
        (DATE(tracking_status_log_min_delivered_date.min_event_date_delivered )) AS tracking_status_log_min_delivered_date_delivered_min_date,
        (DATE(tracking_status_log_min_date_in_transit.min_event_date_in_transit )) AS tracking_status_log_min_date_in_transit_in_transit_min_date,
    network_package.package_id  AS network_package_package_id
FROM network_print_job
LEFT JOIN network_package ON network_print_job.package_id = network_package.package_id
LEFT JOIN pgsql_printcloud_view.info_print_house  AS network_print_house ON network_package.print_house_id = network_print_house.print_house_id
LEFT JOIN network_product_mapping_groups ON network_print_job.product_uid = network_product_mapping_groups.product_uid
LEFT JOIN network_print_job_event ON network_print_job_event.print_job_id = network_print_job.print_job_id
LEFT JOIN tracking_package ON cast(network_package.package_id as string) = tracking_package.PackageID
LEFT JOIN tracking_trail ON tracking_package.ID = tracking_trail.PackageTrackingID
LEFT JOIN tracking_status_log ON tracking_trail.ID = tracking_status_log.AuditTrailID
LEFT JOIN tracking_status_log_min_delivered_date ON tracking_status_log.PackageID = tracking_status_log_min_delivered_date.PackageID
LEFT JOIN tracking_status_log_min_date_in_transit ON tracking_status_log.PackageID = tracking_status_log_min_date_in_transit.PackageID
LEFT JOIN tracking_status_log_min_date_out_for_delivery ON tracking_status_log.PackageID = tracking_status_log_min_date_out_for_delivery.PackageID
LEFT JOIN pgsql_printcloud_view.public_shipment_data  AS network_shipment_data ON network_package.shipment_id = network_shipment_data.shipment_id
LEFT JOIN shipment_method ON network_shipment_data.method_id = shipment_method.method_id
LEFT JOIN pgsql_shipment_service_view.public_providers  AS shipment_provider ON shipment_method.provider_id = shipment_provider.provider_id
INNER JOIN pgsql_shipment_service_view.public_countries  AS shipment_country_to ON network_shipment_data.country_code = shipment_country_to.iso_code
LEFT JOIN shipment_zone_postcode_range ON network_shipment_data.zip >= shipment_zone_postcode_range.postcode_min and
      network_shipment_data.zip <= shipment_zone_postcode_range.postcode_max and
      shipment_country_to.country_id = shipment_zone_postcode_range.country_id and
      shipment_zone_postcode_range.has_postcode_range = true
      --06-Apr-2022 Mahesh:Added for Transit time analysis
      and network_print_house.name = shipment_zone_postcode_range.print_house
      and (case
          when shipment_method.is_enabled = false then concat(shipment_provider.title," (not enabled)")
          else shipment_provider.title end) = shipment_zone_postcode_range.shipment_provider_name
      and shipment_method.title = shipment_zone_postcode_range.shipment_method_name

INNER JOIN shipment_country_zone ON shipment_zone_postcode_range.shipment_country_zone_id = shipment_country_zone.shipment_country_zone_id
      or (
        shipment_country_to.country_id = shipment_country_zone.country_id
        --06-Apr-2022 Mahesh:Added for Transit time analysis
        and network_print_house.name = shipment_country_zone.print_house
        and (case
          when shipment_method.is_enabled = false then concat(shipment_provider.title," (not enabled)")
          else shipment_provider.title end) = shipment_country_zone.shipment_provider_name
        and shipment_method.title = shipment_country_zone.shipment_method_name
        ----
        and (
          network_shipment_data.state_code = shipment_country_zone.administrative_area_level_1 or
          network_shipment_data.city = shipment_country_zone.city or (
            shipment_country_zone.administrative_area_level_1 is null and
            shipment_country_zone.city is null
          )
        ) and
        shipment_country_zone.has_postcode_range = false
      )

LEFT JOIN google_sheets_us.api_country_mapping  AS network_shipment_data_country ON network_shipment_data.country_code = network_shipment_data_country.country_code
INNER JOIN pgsql_shipment_service_view.public_countries  AS shipment_country_from ON network_print_house.country_id = shipment_country_from.iso_code
LEFT JOIN pgsql_shipment_service_view.public_delivery_terms  AS shipment_delivery_terms ON shipment_country_from.iso_code = shipment_delivery_terms.country_from_iso_code and
      shipment_country_to.iso_code = shipment_delivery_terms.country_to_iso_code and
      network_shipment_data.method_id = shipment_delivery_terms.method_id

WHERE ((( case

        when

          (network_print_job_event.shipped_at = network_print_job_event.last_event_at)

          or

          (network_print_job_event.shipped_at >= network_print_job_event.printed_at)

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.shipped_at >= network_print_job_event.prepared_at))

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.prepared_at is null))

          or

          (network_print_job_event.shipped_at <= network_print_job_event.delivered_at)

          or

          (network_print_job_event.shipped_at <= network_print_job_event.in_transit_at)

        then network_print_job_event.shipped_at

        else null

      end

     ) >= ((TIMESTAMP_ADD(TIMESTAMP_TRUNC(CURRENT_TIMESTAMP(), DAY), INTERVAL -1 DAY))) AND ( case

        when

          (network_print_job_event.shipped_at = network_print_job_event.last_event_at)

          or

          (network_print_job_event.shipped_at >= network_print_job_event.printed_at)

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.shipped_at >= network_print_job_event.prepared_at))

          or

          ((network_print_job_event.printed_at is null) and (network_print_job_event.prepared_at is null))

          or

          (network_print_job_event.shipped_at <= network_print_job_event.delivered_at)

          or

          (network_print_job_event.shipped_at <= network_print_job_event.in_transit_at)

        then network_print_job_event.shipped_at

        else null

      end

     ) < ((TIMESTAMP_ADD(TIMESTAMP_ADD(TIMESTAMP_TRUNC(CURRENT_TIMESTAMP(), DAY), INTERVAL -1 DAY), INTERVAL 2 DAY))))) AND ((( tracking_status_log_min_date_in_transit.min_event_date_in_transit  ) IS NOT NULL)) AND (shipment_country_zone.shipment_method_name ) = 'DHL Global Parcel'
GROUP BY
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18
ORDER BY
    6 DESC


  
""", bq_client)

# Prepare df for prediction

In [ ]:
# include (flag) holidays into the data frame 


df_prediction.columns = ['order_day_ind','order_day','PH','Shipment_Method','Product','Shipped_Date','Shipping_Type','Zone','E_Country','Country','Shipping_day_index','Shipping_day','Shipping_hour','Term','out_of_delivery_min_date','delivery_min_date','in_transit_min_date','package_id']
df1 = df_prediction
Country_list = df1['Country'].to_list()
Country_list = list(dict(zip(Country_list,range(0,len(Country_list)))).keys())

from datetime import date
from datetime import timedelta
import holidays
from datetime import datetime
import calendar
from pandas.tseries.offsets import BusinessDay
import numpy as np



Germany1 = []
France1 = []
Netherlands1 = []
Denmark1 = []
Luxembourg1 = []
Italy1 = []
Sweden1 = []
Portugal1 = []
Czechia1 = [] #Czech Republic = []
df1['Country'] = df1['Country'].replace('Czech Republic','Czechia')
Belgium1 = []
Austria1 = []
Bulgaria1 = []
Slovakia1 = [] #Slovak Republic
df1['Country'] = df1['Country'].replace('Slovak Republic','Slovakia')
Ireland1 = []
Canada1 = []
UK1 = []
df1['Country'] = df1['Country'].replace('United Kingdom','UK')
Switzerland1 = []
Greece1 = []
Poland1= []

c_list = ['Poland','Greece','Switzerland','UK','Germany','France','Netherlands','Denmark','Luxembourg','Italy','Sweden','Portugal','Czechia','Belgium','Austria','Bulgaria','Slovakia','Slovakia','Ireland','Canada']

for holiday in holidays.Germany(years=[2019,2020,2021,2022]).items():
  Germany1.append(holiday[0])
for holiday in holidays.France(years=[2019,2020,2021,2022]).items():
  France1.append(holiday[0])
for holiday in holidays.Netherlands(years=[2019,2020,2021,2022]).items():
  Netherlands1.append(holiday[0])
for holiday in holidays.Denmark(years=[2019,2020,2021,2022]).items():
  Denmark1.append(holiday[0])
for holiday in holidays.Luxembourg(years=[2019,2020,2021,2022]).items():
  Luxembourg1.append(holiday[0])
for holiday in holidays.Italy(years=[2019,2020,2021,2022]).items():
  Italy1.append(holiday[0])
for holiday in holidays.Sweden(years=[2019,2020,2021,2022]).items():
  Sweden1.append(holiday[0])
for holiday in holidays.Portugal(years=[2019,2020,2021,2022]).items():
  Portugal1.append(holiday[0])
for holiday in holidays.Czechia(years=[2019,2020,2021,2022]).items():
  Czechia1.append(holiday[0])
for holiday in holidays.Belgium(years=[2019,2020,2021,2022]).items():
  Belgium1.append(holiday[0])
for holiday in holidays.Austria(years=[2019,2020,2021,2022]).items():
  Austria1.append(holiday[0])
for holiday in holidays.Bulgaria(years=[2019,2020,2021,2022]).items():
  Bulgaria1.append(holiday[0])
for holiday in holidays.Ireland(years=[2019,2020,2021,2022]).items():
  Ireland1.append(holiday[0])
for holiday in holidays.Canada(years=[2019,2020,2021,2022]).items():
  Canada1.append(holiday[0])
for holiday in holidays.Poland(years=[2019,2020,2021,2022]).items():
  Poland1.append(holiday[0])
for holiday in holidays.UK(years=[2019,2020,2021,2022]).items():
  UK1.append(holiday[0])
for holiday in holidays.Switzerland(years=[2019,2020,2021,2022]).items():
  Switzerland1.append(holiday[0])
for holiday in holidays.Greece(years=[2019,2020,2021,2022]).items():
  Greece1.append(holiday[0])

class Holidays_c:
  def __init__(self, country):
    self.country = country

Germany = Holidays_c(Germany1)
France = Holidays_c(France1)
Netherlands = Holidays_c(Netherlands1)
Denmark = Holidays_c(Denmark1)
Luxembourg = Holidays_c(Luxembourg1)
Italy = Holidays_c(Italy1)
Sweden = Holidays_c(Sweden1) 
Portugal = Holidays_c(Portugal1)
Czechia = Holidays_c(Czechia1) 
Belgium = Holidays_c(Belgium1)
Austria = Holidays_c(Austria1)
Bulgaria = Holidays_c(Bulgaria1)
Slovakia = Holidays_c(Slovakia1) 
Ireland = Holidays_c(Ireland1)
Canada = Holidays_c(Canada1)
Poland = Holidays_c(Poland1)
Switzerland = Holidays_c(Switzerland1)
Greece = Holidays_c(Greece1)
UK = Holidays_c(UK1)

# include (flag) holidays into the data frame 
def get_holiday(co, sd):
  date_format = "%Y-%m-%d"
  for x in eval(co).country:
    if datetime.strptime(str(x), date_format) == datetime.strptime(str(sd), date_format) + timedelta(days=1):
      h1 = 1
      break
  else:
    h1 = 0
  return h1


def add_holidays(co, sd):
  if co in c_list:
    h11 = get_holiday(co,sd)
  else:
    h11 = int(0)
  return h11

def get_holiday2(co, sd):
  date_format = "%Y-%m-%d"
  for x in eval(co).country:
    if datetime.strptime(str(x), date_format) == datetime.strptime(str(sd), date_format) + timedelta(days=2):
      h1 = 1
      break
  else:
    h1 = 0
  return h1


def add_holidays2(co, sd):
  if co in c_list:
    h11 = get_holiday2(co,sd)
  else:
    h11 = int(0)
  return h11

def get_holiday3(co, sd):
  date_format = "%Y-%m-%d"
  for x in eval(co).country:
    if datetime.strptime(str(x), date_format) == datetime.strptime(str(sd), date_format) + timedelta(days=3):
      h1 = 1
      break
  else:
    h1 = 0
  return h1


def add_holidays3(co, sd):
  if co in c_list:
    h11 = get_holiday3(co,sd)
  else:
    h11 = int(0)
  return h11

def get_holiday4(co, sd):
  date_format = "%Y-%m-%d"
  for x in eval(co).country:
    if datetime.strptime(str(x), date_format) == datetime.strptime(str(sd), date_format) + timedelta(days=4):
      h1 = 1
      break
  else:
    h1 = 0
  return h1


def add_holidays4(co, sd):
  if co in c_list:
    h11 = get_holiday4(co,sd)
  else:
    h11 = int(0)
  return h11

df1['holiday1'] = df1.apply(lambda x: add_holidays(x.Country,x.Shipped_Date), axis = 1)
df1['holiday2'] = df1.apply(lambda x: add_holidays2(x.Country,x.Shipped_Date), axis = 1)
df1['holiday3'] = df1.apply(lambda x: add_holidays3(x.Country,x.Shipped_Date), axis = 1)
df1['holiday4'] = df1.apply(lambda x: add_holidays4(x.Country,x.Shipped_Date), axis = 1)


df1['Country_PH'] = df1['PH'].apply(lambda x: 'Germany' if x[0:2] == 'DE' else ('Czechia' if x[0:2] == 'CZ' else 'None'))

In [ ]:
df_pre = df1[['PH', 'Shipping_Type', 'Zone', 'Country', 'order_day_ind',
       'Shipping_day_index', 'holiday1', 'holiday2', 'holiday3', 'holiday4',
       'Shipping_hour','Product']]
      

# Prediction


In [ ]:
import pickle
from joblib import dump, load


with open('/content/drive/MyDrive/Models/model_v1.pkl', 'rb') as file:
    s = pickle.load(file)

y_new_pre = s.predict(df_pre)

# Data Table with Prediction for BigQuery

In [ ]:
import math
df_prediction['Prediction'] = [math.ceil(x) for x in list(y_new_pre)]
df_prediction_BQ = df_prediction[['package_id','Prediction']]
prediction_late_packages_csv = df_prediction_BQ.to_csv(index = False)
